<a href="https://colab.research.google.com/github/Natural-Language-Processing-YU/Module-1-Assignment/blob/main/M1_Assignment_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M1 Assignment: Text Processing and Edit Distance

In this section we will be exploring how to preprocess tweets . We will provide a function for preprocessing tweets during this week's assignment, but it is still good to know what is going on under the hood. By the end of this assignment, you will see how to use the [NLTK](http://www.nltk.org) package to perform a preprocessing pipeline for Twitter datasets.

## Setup
Eventually, you will conduct a sentiment analysis on Tweets. To help with that, we will be using the Natural Language Toolkit (NLTK) package, an open-source Python library for natural language processing. In this library you will use the NLTK to assist with processing the tweet to clean it for interpretation.

In Part 1.1, you will extract a set of Elon Musk tweets. Next in Part 1.2, you will process the tweets using the various processing tasks outlined in the section and Chapter 2 of Jurafsky and Martin. Finally, in Part 1.3, you will create a simple version of a Levensthein distance formula to run the edit distance between two matrices.

As part of completing the assignment, you will see that there are areas in the note book for you to complete your own coding input.

It will be look like following:
```
### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
'Some coding activity for you to complete'
### END CODE HERE ###

```

Additionally, you will be using custom media and libraries created and stored for your in your Colab environment.


## Part 1.1: Using tweetsts

---



### Extracting tweets
In this next section, we're going to import Elon Musk's tweets and prepare them to preprocess. Use the following code to connect to the M1 Assignment Repo and download it to your Google Colab Account



In [3]:
#install this if you do not have this already installed
#!pip install gitpython

import git
import sys


# Clone the GitHub repository
repo_url = 'https://github.com/Natural-Language-Processing-YU/Module-1-Assignment.git'
repo_dir = '/content/m1_repo'  # Specify the directory to clone the repository
git.Repo.clone_from(repo_url, repo_dir)
# Add the cloned repository directory to the import path



<git.repo.base.Repo '/content/m1_repo/.git'>

Use the pandas library to create a dataframe with the csv file full owf twees.

In [4]:
#extract the tweets
import pandas as pd

tweets = pd.read_csv('/content/m1_repo/data/elonmusk_tweets.csv') #import file
print(tweets.dtypes) #print data types
print(tweets.text) #show tweets from file
df = pd.DataFrame(tweets)



id             int64
created_at    object
text          object
dtype: object
0       b'And so the robots spared humanity ... https:...
1       b"@ForIn2020 @waltmossberg @mims @defcon_5 Exa...
2           b'@waltmossberg @mims @defcon_5 Et tu, Walt?'
3                     b'Stormy weather in Shortville ...'
4       b"@DaveLeeBBC @verge Coal is dying due to nat ...
                              ...                        
2814                 b'That was a total non sequitur btw'
2815    b'Great Voltaire quote, arguably better than T...
2816    b'I made the volume on the Model S http://t.co...
2817    b"Went to Iceland on Sat to ride bumper cars o...
2818    b'Please ignore prior tweets, as that was some...
Name: text, Length: 2819, dtype: object


## Part 1.2: Preprocessing the text from tweets
Text processing is one of the critical steps in an NLP project and in data scenience and analytics. It includes cleaning and formatting the data before feeding an algorithm. For NLP, the preprocessing steps are comprised of the following tasks:

1. Tokenizing the string
2. Lowercasing
3. Removing stop words and punctuation
4. Stemming

We will take this approach with a selected tweet that we returned from above see how this is transformed by each preprocessing step.

Let's take one of the tweets and apply preprocessing steps

Let's take a look at one tweet from our dataframe:

In [6]:
print('\033[92m' + df['text'][2816])

b'I made the volume on the Model S http://t.co/wMCnT53M go to 11.  Now I just need to work in a miniature Stonehenge...'


### Remove hyperlinks, hashtags, and beginngin of strings
Using this tweet, let's clean it up to remove unncessary information. First, we will use regex to remove hyperlinks. You will create the regex substring to remove hyperlinks, hashtabs and the start of a tweet line.  

In [10]:
### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
#create regex for hyperlinks

import re

# Disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None

'''
# Below regex pattern matches most URLs, including those that start with http or https,
# followed by any combination of alphanumeric characters and some special characters.
'''
regex_remove_hyperlinks = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
df['text'][2816] = re.sub(regex_remove_hyperlinks, '', df['text'][2816])

# remove hashtags
# only removing the hash # sign from the word
'''
# Below regex pattern finds hashtags (words preceded by a # sign) and captures the word
# following the hash without the hash itself. The replacement pattern \1 refers to
# the first captured group, which is the word without the hash, effectively removing
# the hash sign while keeping the word.
'''
regex_hash = r"#(\w+)"
df['text'][2816] = re.sub(regex_hash, '', df['text'][2816])


# remove 'b from each string
regex_string_beginning = r"^b['\"]"
df['text'][2816] = re.sub(regex_string_beginning, '', df['text'][2816])

### END CODE HERE ###


# Print the modified tweet
print('\033[92m' + df['text'][2816])



I made the volume on the Model S  go to 11.  Now I just need to work in a miniature Stonehenge...'


#Using NLTK Libraries

Now let's use the NLTK libraries to remove stopwords, tokenize, and stem the words.

The Porter stemming algorithm, also known as the Porter stemmer, is a widely used algorithm for stemming words in natural language processing (NLP). It is named after its creator, Martin Porter. The goal of stemming is to reduce words to their base or root form, which helps to normalize variations of a word and reduce the vocabulary size.

In [11]:
import nltk                                # Python library for NLP
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

In [12]:
# download the stopwords from NLTK
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Tokenize the string

To tokenize means to split the strings into individual words without blanks or tabs. In this same step, we will also convert each word in the string to lower case. The [tokenize](https://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize.casual) module from NLTK allows us to do these easily:

In [13]:
# instantiate tokenizer class
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)

# tokenize tweets
tokenized_tweet = tokenizer.tokenize(str(df['text'][2816]))


print(tokenized_tweet)

['i', 'made', 'the', 'volume', 'on', 'the', 'model', 's', 'go', 'to', '11', '.', 'now', 'i', 'just', 'need', 'to', 'work', 'in', 'a', 'miniature', 'stonehenge', '...', "'"]


### Remove stop words and punctuations

The next step is to remove stop words and miscelleneous punctuations. Stop words are words that do not have semantic meaning to the tweet. There is a library of stopwords built into NLTK. The list provided by NLTK when you run the cells below.

In [14]:
#Import the english stop words list from NLTK
stopwords_english = stopwords.words('english')

print('Stop words\n')
print(stopwords_english)

print('\nPunctuation\n')
print(string.punctuation)

Stop words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so

We can see that the stop words list above contains some words that could be important in some contexts.
These could be words like _i, not, between, because, won, against_. In some cases, you may want to update this dictionary of stop words to suit your needs.

Certain groupings like ':)' and '...'  should be retained when dealing with tweets because they are used to express emotions, but in some cases they should be removed.


In [16]:
df2=df
tweets_clean = []

for word in tokenized_tweet: # Go through every word in your tokens list
    if (word not in stopwords_english and  # remove stopwords
        word not in string.punctuation):  # remove punctuation
        tweets_clean.append(word)

print('removed stop words and punctuation:')
print(tweets_clean)

removed stop words and punctuation:
['made', 'volume', 'model', 'go', '11', 'need', 'work', 'miniature', 'stonehenge', '...']


Next, we use porter stemmer to stem words.

            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

In [17]:
import nltk
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
for word in tokenized_tweet:
    stem_word = stemmer.stem(word)
    tweets_clean.append(stem_word)
print(tweets_clean)

['made', 'volume', 'model', 'go', '11', 'need', 'work', 'miniature', 'stonehenge', '...', 'i', 'made', 'the', 'volum', 'on', 'the', 'model', 's', 'go', 'to', '11', '.', 'now', 'i', 'just', 'need', 'to', 'work', 'in', 'a', 'miniatur', 'stoneheng', '...', "'"]


## preprocess_tweet()

As shown above, preprocessing consists of multiple steps before you arrive at the final list of words.  In the week's assignment, write a function called `preprocess_tweet()`.

Then, use this to iterate through the dataframe and preprocess each tweet into a new column.

Print your results.


In [26]:
### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
import nltk
from nltk.stem import PorterStemmer

def preprocess_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    # Step 1: Remove hyperlinks from tweeet
    regex_remove_hyperlinks = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    tweet = re.sub(regex_remove_hyperlinks, '', tweet)

    # Step 2: remove hashtags
    regex_hash = r"#(\w+)"
    tweet = re.sub(regex_hash, '', tweet)

    # Step 3: remove 'b from each string
    regex_string_beginning = r"^b['\"]"
    tweet = re.sub(regex_string_beginning, '', tweet)

    # Step 4: Tokenize the input 'tweet' string
    # instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)

    # tokenize tweets
    tokenized_tweet = tokenizer.tokenize(tweet)

    # Step 5: Remove stopwords and punctuations
    stopwords_english = stopwords.words('english')
    punctuations = string.punctuation

    tweets_clean = []
    for word in tokenized_tweet: # Go through every word in your tokens list
      if (word not in stopwords_english and  # remove stopwords
          word not in string.punctuation):  # remove punctuation
          tweets_clean.append(word)

    # Step 6: Use porter stemmer to stem words
    final_clean_list = []
    stemmer = PorterStemmer()
    for word in tweets_clean:
      stem_word = stemmer.stem(word)
      final_clean_list.append(stem_word)

    return final_clean_list
### END CODE HERE ###

In [27]:
### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
"""
Use the function to iterate through the original dataframe 'df' and preprocess each tweet into a new column "preprocessed_tweets". Pseudocode below.

for i in len(df):
  df["preprocessed_tweet"][i] = preprocess_tweet(df["text"][i])


"""
df["preprocessed_tweet"] = df["text"].apply(preprocess_tweet)

### END CODE HERE ###

display(df)

,id,created_at,text,preprocessed_tweet
0,849636868052275200,2017-04-05 14:56:29,b'And so the robots spared humanity ... https:...,"[robot, spare, human, ...]"
1,848988730585096192,2017-04-03 20:01:01,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa...","[exactli, tesla, absurdli, overvalu, base, pas..."
2,848943072423497728,2017-04-03 16:59:35,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'","[et, tu, walt]"
3,848935705057280001,2017-04-03 16:30:19,b'Stormy weather in Shortville ...',"[stormi, weather, shortvil, ...]"
4,848416049573658624,2017-04-02 06:05:23,"b""@DaveLeeBBC @verge Coal is dying due to nat ...","[coal, die, due, nat, ga, frack, basic, dead]"
...,...,...,...,...
2814,142881284019060736,2011-12-03 08:22:07,b'That was a total non sequitur btw',"[total, non, sequitur, btw]"
2815,142880871391838208,2011-12-03 08:20:28,"b'Great Voltaire quote, arguably better than T...","[great, voltair, quot, arguabl, better, twain,..."
2816,142188458125963264,2011-12-01 10:29:04,I made the volume on the Model S go to 11. N...,"[made, volum, model, go, 11, need, work, minia..."
2817,142179928203460608,2011-12-01 09:55:11,"b""Went to Iceland on Sat to ride bumper cars o...","[went, iceland, sat, ride, bumper, car, ice, c..."


from matplotlib import pyplot as plt
df['id'].plot(kind='hist', bins=20, title='id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['id'].plot(kind='line', figsize=(8, 4), title='id')
plt.gca().spines[['top', 'right']].set_visible(False)

## Part 1.3 Create a Levensthein Distance Formula

Recall that Edit Distance is the similarity between two words represented numericallly. Levenstehin distance is one of the most common algorithms used in calculating the edit distance between two words.

Create your own simple Levensthein distance function. Then return the results of the distance between two words: _stemming_ and _lemmatization_.


In [7]:
def leven_dist(string1, string2):
    '''
    input:
        string1 = the first word in your formula
        string2 = the second word in your formula
    output:
        levenschtein edit distance

    '''
    #if min(i,j) =/= 0
    if not string1: return len(string2)
    if not string2: return len(string1)

### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    #because min(i,j) =/= 0 then we min(i,j)

    return min(

        #part I. calculate the numerical position of letter i-1, j and add 1
        leven_dist(string1[:-1], string2) + 1,


        #part II: calculate the numerical position of letter i, j-1 and add 1
        leven_dist(string1, string2[:-1]) + 1,

        # part III: if position i-1, j-1 are not the same letter, then add 1
        leven_dist(string1[:-1], string2[:-1]) +
         (0 if string1[-1] == string2[-1] else 1)
    )
### END CODE HERE ###



#now run your results

string1 = 'stemming'
string2 = 'lemmatization'
print("Your Levensthein Distance is: ",leven_dist(string1,string2))

Your Levensthein Distance is:  10


### Expected output:
Your Levensthein Distance is: 10

#end of assignment#
                                            
Source:
Natural Language preprocessing, deeplearning.ai
Twitter API documentation
Wikipedia: Levensthein Distance
Chapter 2 (Jurafsky and Martin)
                        
